In [36]:
import pandas as pd
import numpy as np

In [61]:
train=pd.read_csv('../data/train.csv')
test=pd.read_csv('../data/test.csv')
# combine test dataset and train dataset
df =pd.concat([train, test], axis=0)

In [62]:
# remove unknown
df_no_unknown = df[(df['job'] != 'unknown') & (df['education'] != 'unknown') & (df['marital'] != 'unknown') & (df['default'] != 'unknown') & (df['housing'] != 'unknown') & (df['loan'] != 'unknown')]
# filter previous success cases
df_prev_success = df_no_unknown[df_no_unknown['poutcome'] == 'success']

In [90]:
train_df = pd.DataFrame()
train_df['default'] = df_prev_success['default'].map({'yes': True, 'no': False})
train_df['loan'] = df_prev_success['loan'].map({'yes': True, 'no': False})
train_df['housing'] = df_prev_success['housing'].map({'yes': True, 'no': False})

In [91]:
features_transform = ['job', 'education', 'marital',
                      'contact','month','day_of_week']

for feature in features_transform:
    feature_df =  pd.get_dummies(df_prev_success[feature])
    train_df = pd.concat([train_df.reset_index(drop=True), feature_df.reset_index(drop=True)], axis=1)

      admin.  blue-collar  entrepreneur  housemaid  management  retired  \
4       True        False         False      False       False    False   
10     False        False         False      False       False    False   
34      True        False         False      False       False    False   
45     False        False         False      False       False    False   
59     False        False         False      False       False     True   
...      ...          ...           ...        ...         ...      ...   
7462    True        False         False      False       False    False   
7466   False         True         False      False       False    False   
7474   False        False         False      False       False    False   
7488   False        False         False      False       False    False   
7499    True        False         False      False       False    False   

      self-employed  services  student  technician  unemployed  
4             False     False    F

In [92]:
numeric_X = df_prev_success[['age', 'campaign', 'pdays', 'previous']]
y = df_prev_success['duration']

In [93]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
numeric_X = pd.DataFrame(scaler.fit_transform(numeric_X), columns = numeric_X.columns)

In [94]:
Xnew = pd.concat([train_df, numeric_X], axis=1)

In [95]:
# import train_test split
from sklearn.model_selection import train_test_split
# split data into train and test, 80% in training and 20% in testing
X_train, X_test, y_train, y_test = train_test_split(Xnew, y, test_size=0.2,
                                                    random_state = 99)

In [96]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression()

In [97]:
coeff_df = pd.DataFrame(lin_reg.coef_, Xnew.columns, columns=['Coefficient'])
coeff_df

,Coefficient
default,-4.865803e+01
loan,1.673889e+01
housing,-3.425443e+01
admin.,-2.193496e+16
blue-collar,-2.193496e+16
entrepreneur,-2.193496e+16
housemaid,-2.193496e+16
management,-2.193496e+16
retired,-2.193496e+16
self-employed,-2.193496e+16
